In [1]:
import torch
import lightning as L
import numpy as np
import cv2 as cv
from pathlib import Path
import matplotlib.pyplot as plt
import torchvision.models as models

from image import Image
from hooked_vgg import Hooked_VGG

In [ ]:
img = Image(Path('./test_images/wolf_forest.jpg'))
vgg = Hooked_VGG()

In [3]:
vgg.vgg

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [126]:
vgg = Hooked_VGG([0,1,2,3])
print(vgg.hooks.keys())
vgg.add_hooks([5,6])
print(vgg.hooks.keys())
vgg.add_hooks(4)
print(vgg.hooks.keys())
feat1 = vgg.get_features(Image(Path('./test_images/wolf_forest.jpg')).to_tensor())
print(vgg.hooks.keys())
vgg.remove_hooks(4)
feat2 = vgg.get_features(Image(Path('./test_images/wolf_forest.jpg')).to_tensor())
vgg.remove_hooks([0,2])
feat3 = vgg.get_features(Image(Path('./test_images/wolf_forest.jpg')).to_tensor())

dict_keys([0, 1, 2, 3])
dict_keys([0, 1, 2, 3, 5, 6])
dict_keys([0, 1, 2, 3, 5, 6, 4])
dict_keys([0, 1, 2, 3, 5, 6, 4])


In [124]:
import timeit

vgg = Hooked_VGG([0,1,2,3]).to("cuda")
img = Image(Path('./test_images/wolf_forest.jpg')).to("cuda")
img_t = img.to_tensor()

%timeit vgg.get_features(img_t)

vgg.to("cpu")
img.to("cpu")
img_t = img.to_tensor()

%timeit vgg.get_features(img_t)

18.9 ms ± 175 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
445 ms ± 8.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [118]:
feat1[0].device

device(type='cuda', index=0)

In [105]:
vgg2 = Hooked_VGG()
vgg2.add_hooks([0,1])
feat4 = vgg2.get_features(Image(Path('./test_images/wolf_forest.jpg')).to_tensor())
feat4.keys()

dict_keys([0, 1])

In [82]:
feat3.keys()

dict_keys([3, 6])